In [42]:
import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets


def custom_loader(batch_size, shuffle_test=False, data_path='./dataset/preprocessed/train/'):
    # Add the necessary transforms
    # normalize = transforms.Normalize(mean=[0.024], std=[0.994])
    transform = transforms.Compose([
        transforms.Resize((48, 48)),  # Adjust this if your images are a different size
        # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.ToTensor(),
        # normalize
    ])

    # Load your dataset using ImageFolder
    master_dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Calculate the sizes of the splits
    total_size = len(master_dataset)
    train_size = int(0.85 * total_size)
    val_size = total_size - train_size

    # Use random_split to create datasets for training, testing, and validation
    train_dataset, val_dataset = random_split(master_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader
    
class MultiLayerFCNet(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()

        self.layer1=nn.Conv2d(3,32,4,padding=1,stride=1)
        self.B1 = nn.BatchNorm2d(32)
        self.layer2 = nn.Conv2d(32, 32, 4, padding=1, stride=1)
        self.B2 = nn.BatchNorm2d(32)
        self.Maxpool=nn.MaxPool2d(2)
        self.layer3 = nn.Conv2d(32, 64, 4, padding=1, stride=1)
        self.B3 = nn.BatchNorm2d(64)
        self.layer4 = nn.Conv2d(64, 64, 4, padding=1, stride=1)
        self.B4 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.5)
        
        # New layers
        self.layer5 = nn.Conv2d(64, 128, 4, padding=1, stride=1)
        self.B5 = nn.BatchNorm2d(128)
        self.layer6 = nn.Conv2d(128, 128, 4, padding=1, stride=1)
        self.B6 = nn.BatchNorm2d(128)
        self.layer7 = nn.Conv2d(128, 256, 4, padding=1, stride=1)
        self.B7 = nn.BatchNorm2d(256)
        self.layer8 = nn.Conv2d(256, 256, 4, padding=1, stride=1)
        self.B8 = nn.BatchNorm2d(256)
        
        # Calculate the size for the fully connected layer after additional max-pooling layers
        # Assuming two max-pooling operations in the existing layers
        self.fc_size = 256   # Now this is 256 * 3 * 3
        self.fc = nn.Linear(self.fc_size, output_size)

    def forward(self, x):
        # Pass through existing layers
        x = F.leaky_relu(self.B1(self.layer1(x)))
        x = self.Maxpool(F.leaky_relu(self.B2(self.layer2(x))))
        x = F.leaky_relu(self.B3(self.layer3(x)))
        x = self.Maxpool(F.leaky_relu(self.B4(self.layer4(x))))
        
        # Pass through new layers
        x = F.leaky_relu(self.B5(self.layer5(x)))
        x = F.leaky_relu(self.B6(self.layer6(x)))
        x = self.Maxpool(F.leaky_relu(self.B7(self.layer7(x))))
        x = self.Maxpool(F.leaky_relu(self.B8(self.layer8(x))))
        
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        return self.fc(x)

if __name__ == '__main__':

    batch_size = 64
    test_batch_size = 64
    input_size = 3 * 48 * 48  # 1 channels, 48x48 image size
    hidden_size = 50  # Number of hidden units
    output_size = 4  # Number of output classes 4
    num_epochs = 10

    # train_loader, _ = cifar_loader(batch_size)
    # _, test_loader = cifar_loader(test_batch_size)
    train_loader, test_loader = custom_loader(batch_size, data_path='./dataset/preprocessed/train/')
    # dataloader = DataLoader(dataset=IrisDataset('iris.data'),
    #                         batch_size=10,
    #                         shuffle=True)

    epochs = 50
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = MultiLayerFCNet(input_size, hidden_size, output_size)
    model = nn.DataParallel(model)
    model.to(device)
    #model.load_state_dict(torch.load('path'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    BestACC=0.3
    for epoch in range(epochs):
        running_loss = 0
        for instances, labels in train_loader:
            optimizer.zero_grad()

            output = model(instances)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(running_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            allsamps=0
            rightPred=0

            for instances, labels in test_loader:

                output = model(instances)
                predictedClass=torch.max(output,1)
                allsamps+=output.size(0)
                rightPred+=(torch.max(output,1)[1]==labels).sum()


            ACC=rightPred/allsamps
            print("epoch=",epoch)
            print('Accuracy is=',ACC*100)
            #if the acc is greater than the best acc, save the model
            if ACC>BestACC:
                torch.save(model.state_dict(), './model/best_model.pth')
                BestACC=ACC
        model.train()



0.5599958506123773
epoch= 0
Accuracy is= tensor(75.7669)
0.5211039007395163
epoch= 1
Accuracy is= tensor(75.7669)
0.5093552065991808
epoch= 2
Accuracy is= tensor(75.7669)


KeyboardInterrupt: 

In [35]:
import torch
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 'MultiLayerFCNet' is the model class
model = MultiLayerFCNet(input_size, hidden_size, output_size)


# If the model was originally wrapped in nn.DataParallel for training, do the same here
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

# Load the model weights
model.load_state_dict(torch.load('./model/best_model.pth'))

# Move the model to the appropriate device
model = model.to(device)

model.eval()  # Set the model to evaluation mode

def custom_loader(batch_size, shuffle_test=False, data_path='./dataset_split'):
    # Add the necessary transforms
    # normalize = transforms.Normalize(mean=[0.024], std=[0.994])
    # transform = transforms.Compose([
    #     transforms.Resize((48, 48)),  # Adjust this if your images are a different size
    #     # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    #     transforms.ToTensor(),
    #     # normalize
    # ])

    # Load your custom dataset
    test_dataset = datasets.ImageFolder(root=data_path + '/Test')

    # Data loaders
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True)

    return  test_loader


# Prepare your test loader
test_loader = custom_loader(batch_size, shuffle_test=True, data_path='./dataset_split')

# Initialize lists to track predictions and true labels
all_preds = []
all_labels = []

# No gradients needed for evaluation
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predictions
        _, preds = torch.max(outputs, 1)

        # Store predictions and true labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute the confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title('Confusion Matrix')
plt.show()


RuntimeError: Error(s) in loading state_dict for MultiLayerFCNet:
	Missing key(s) in state_dict: "layer1.weight", "layer1.bias", "B1.weight", "B1.bias", "B1.running_mean", "B1.running_var", "layer2.weight", "layer2.bias", "B2.weight", "B2.bias", "B2.running_mean", "B2.running_var", "layer3.weight", "layer3.bias", "B3.weight", "B3.bias", "B3.running_mean", "B3.running_var", "layer4.weight", "layer4.bias", "B4.weight", "B4.bias", "B4.running_mean", "B4.running_var", "layer5.weight", "layer5.bias", "B5.weight", "B5.bias", "B5.running_mean", "B5.running_var", "layer6.weight", "layer6.bias", "B6.weight", "B6.bias", "B6.running_mean", "B6.running_var", "layer7.weight", "layer7.bias", "B7.weight", "B7.bias", "B7.running_mean", "B7.running_var", "layer8.weight", "layer8.bias", "B8.weight", "B8.bias", "B8.running_mean", "B8.running_var", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "module.layer1.weight", "module.layer1.bias", "module.B1.weight", "module.B1.bias", "module.B1.runni